# 📓 01 — Exploratory Data Analysis (EDA)

**Exoplanet Habitability Scorer**

---

## 1️⃣ Notebook Setup & Context

**To do**

* Add a short markdown cell explaining:

  * what dataset is being used (`Planetary Systems Composite`)
  * why EDA is required before feature engineering
* Import core libraries:

  * `pandas`, `numpy`
  * `matplotlib`, `seaborn`
  * `missingno` (optional but great)

**Goal**

> Establish context and make the notebook readable in isolation.

---

## 2️⃣ Load & Inspect Raw Data

**To do**

* Load CSV (skip metadata comment lines)
* Display:

  * shape of the dataframe
  * column names
  * first 5 rows

**Checks**

* Confirm planet-level granularity (one row per planet)
* Confirm presence of value + error + limit columns

**Goal**

> Verify assumptions about data structure early.

---

## 3️⃣ Identify Candidate Habitability Features

**To do**

* Create a shortlist of:

  * planetary features
  * stellar features
* Explicitly ignore non-relevant metadata (discovery method, references, etc.)

**Example output**

```text
Planetary:
- pl_rade
- pl_bmasse
- pl_orbper
- pl_orbeccen
- pl_eqt

Stellar:
- st_teff
- st_rad
```

**Goal**

> Narrow scope and avoid feature creep.

---

## 4️⃣ Missing Data Analysis

**To do**

* Calculate % missing for each shortlisted feature
* Visualise missingness:

  * bar chart
  * missingno matrix (optional)
* Identify:

  * features with acceptable missingness
  * features likely to require inference or exclusion

**Questions to answer**

* How many planets remain if we require radius + mass?
* Is missingness random or systematic?

**Goal**

> Decide what can be realistically used in v1.

---

## 5️⃣ Data Type & Value Sanity Checks

**To do**
For each key feature:

* Confirm dtype is numeric
* Check min / max values
* Identify impossible values:

  * negative mass or radius
  * extreme temperatures
  * eccentricity > 1

**Action**

* Flag rows for removal (don’t remove yet — just count)

**Goal**

> Catch silent data quality issues.

---

## 6️⃣ Distribution Analysis (Raw Values)

**To do**
For each shortlisted feature:

* Plot distribution (histogram / KDE)
* Use log-scale where appropriate (e.g. mass, period)
* Annotate Earth reference values

**Questions to answer**

* Are distributions heavily skewed?
* Where does Earth sit relative to exoplanets?

**Goal**

> Build intuition before normalisation.

---

## 7️⃣ Outlier Identification (Not Removal Yet)

**To do**

* Identify extreme values using:

  * percentiles
  * IQR
* Compare:

  * genuine astronomical extremes
  * likely measurement artefacts

**Important**
❌ Do NOT blindly drop outliers
✅ Document them

**Goal**

> Separate “rare but real” from “probably wrong”.

---

## 8️⃣ Correlation & Redundancy Checks

**To do**

* Compute correlation matrix for candidate features
* Visualise correlations (heatmap)
* Identify:

  * highly correlated pairs
  * redundant variables

**Examples**

* Mass vs radius
* Stellar temperature vs equilibrium temperature

**Goal**

> Avoid leaking duplicate information into the model later.

---

## 9️⃣ Error & Uncertainty Exploration 

**To do**

* Inspect presence of error columns:

  * `*_err1`, `*_err2`
* Plot:

  * distribution of relative errors
* Identify features with:

  * very high uncertainty
  * frequent upper/lower limits

**Goal**

> Decide where uncertainty-aware scoring is most valuable.

---

## 🔟 Initial Filtering Decision

**To do**

* Define **v1 inclusion rules**, e.g.:

  * must have planet radius
  * must have planet mass
  * allow missing equilibrium temperature
* Count how many planets survive

**Output**

* Before vs after row counts
* % data retained

**Goal**

> Make filtering transparent and reproducible.

---

## 1️⃣1️⃣ Key Takeaways

**To do**
Add a markdown cell summarising:

* which features are viable
* main data limitations
* what will be handled in feature engineering vs modelling

**Example**

> “Although equilibrium temperature is missing for ~25% of planets, radius and mass coverage is high, allowing a robust Earth-normalised baseline score.”


---

## 1️⃣2️⃣ EDA Exit Criteria (Final Cell)

**To do**
Explicitly state:

* which features move forward
* what assumptions have been made
* what remains unresolved

**Goal**

> Clean handoff to `02_feature_engineering.ipynb`.

## 1.1 Dataset overview

We use the **Planetary Systems Composite** dataset (NASA Exoplanet Archive) from the local CSV in the `data/` folder. This notebook performs EDA to verify structure, coverage, and data quality **before** feature engineering, so later steps are grounded in validated assumptions.

In [1]:
# Core libraries
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Optional: missingness visualization
try:
    import missingno as msno
    MISSINGNO_AVAILABLE = True
except Exception:
    MISSINGNO_AVAILABLE = False

sns.set_theme(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)

Matplotlib is building the font cache; this may take a moment.


In [2]:
# Load raw data (skip metadata/comment lines if present)
data_path = "../data/raw_data.csv"

df = pd.read_csv(data_path, comment="#")

# Basic inspection
print("Shape:", df.shape)
print("Columns:", len(df.columns))
display(df.head())

# Quick check: column name samples
pd.Series(df.columns).head(20)

Shape: (6087, 169)
Columns: 169


,rowid,pl_name,hostname,pl_letter,hd_name,hip_name,tic_id,gaia_dr2_id,gaia_dr3_id,sy_snum,...,sy_tmag_reflink,sy_kepmag,sy_kepmag_reflink,pl_nnotes,st_nphot,st_nrvc,st_nspec,pl_nespec,pl_ntranspec,pl_ndispec
0,1,11 Com b,11 Com,b,HD 107383,HIP 60202,TIC 72437047,Gaia DR2 3946945413106333696,Gaia DR3 3946945413106333696,2,...,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,NaN,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,2.0,1.0,2.0,0.0,0.0,0.0,0.0
1,2,11 UMi b,11 UMi,b,HD 136726,HIP 74793,TIC 230061010,Gaia DR2 1696798367260229376,Gaia DR3 1696798367260229376,1,...,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,NaN,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,0.0,1.0,1.0,0.0,0.0,0.0,0.0
2,3,14 And b,14 And,b,HD 221345,HIP 116076,TIC 333225860,Gaia DR2 1920113512486282240,Gaia DR3 1920113512486282240,1,...,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,NaN,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,0.0,1.0,1.0,0.0,0.0,0.0,0.0
3,4,14 Her b,14 Her,b,HD 145675,HIP 79248,TIC 219483057,Gaia DR2 1385293808145621504,Gaia DR3 1385293808145621504,1,...,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,NaN,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,0.0,1.0,4.0,1.0,0.0,0.0,0.0
4,5,16 Cyg B b,16 Cyg B,b,HD 186427,HIP 96901,TIC 27533327,Gaia DR2 2135550755683407232,Gaia DR3 2135550755683407232,3,...,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,6.095,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,5.0,1.0,4.0,3.0,0.0,0.0,0.0


0               rowid
1             pl_name
2            hostname
3           pl_letter
4             hd_name
5            hip_name
6              tic_id
7         gaia_dr2_id
8         gaia_dr3_id
9             sy_snum
10            sy_pnum
11            sy_mnum
12            cb_flag
13    discoverymethod
14          disc_year
15       disc_refname
16       disc_pubdate
17        disc_locale
18      disc_facility
19     disc_telescope
dtype: str